In [1]:
from tax_inspector_competition import CompetitionSimulator, DayRoute, POI
from typing import List, Dict, Tuple
from csv_starting_points import *
import pandas

class Traveler: 
    def __init__(self, dataset_path: str):
        self.simulator = CompetitionSimulator(dataset_path)
        self.inspectors = []
        self.competition_results = {}
        self.fixed_starting_points = None
    def _high_value_strategy(self, starting_point: POI) -> List[POI]:
        """Strategia che privilegia POIs con valore alto"""
        jurisdiction_pois = self.simulator.optimizer.jurisdictions.get(starting_point.jurisdiction, [])
        available_pois = [poi for poi in jurisdiction_pois if poi.id != starting_point.id]
        
        # Ordina per valore decrescente
        available_pois.sort(key=lambda poi: poi.fee_value, reverse=True)
        
        # Prova ad aggiungere POIs in ordine di valore
        selected_pois = []
        for poi in available_pois:
            test_route = selected_pois + [poi]
            if (len(test_route) <= 8 and 
                self.simulator.optimizer.is_valid_route(starting_point, test_route)):
                selected_pois.append(poi)
        
        return selected_pois

In [2]:
t = Traveler("dataset_finale.csv")

In [3]:
df_dum = pd.read_csv('coordinate_per_giorno.csv')
df_dum.columns = ["day", "lat", "lon"]
df_dum.to_csv("st_points.csv")
starting_points = load_starting_points_from_coordinates_csv('st_points.csv')

if starting_points:
    print(f"\nDettagli punti di partenza:")
    for i, poi in enumerate(starting_points, 1):
        print(f"  Giorno {i}: ({poi.lat}, {poi.lon}) -> {poi.jurisdiction}")

Caricati 5 punti di partenza da 'st_points.csv'
Distribuzione per giurisdizione:
   J2: 1 punti
   J4: 1 punti
   J5: 2 punti
   J6: 1 punti

Dettagli punti di partenza:
  Giorno 1: (41.8907406, 12.4773952) -> J5
  Giorno 2: (41.9023265, 12.4988301) -> J6
  Giorno 3: (41.882561, 12.514131) -> J4
  Giorno 4: (41.8970256, 12.4744365) -> J5
  Giorno 5: (41.8461723, 12.4879358) -> J2


In [4]:
starting_points

[POI(id=9000, lat=41.8907406, lon=12.4773952, poi_type='starting_point', fee_value=0.0, jurisdiction='J5'),
 POI(id=9001, lat=41.9023265, lon=12.4988301, poi_type='starting_point', fee_value=0.0, jurisdiction='J6'),
 POI(id=9002, lat=41.882561, lon=12.514131, poi_type='starting_point', fee_value=0.0, jurisdiction='J4'),
 POI(id=9003, lat=41.8970256, lon=12.4744365, poi_type='starting_point', fee_value=0.0, jurisdiction='J5'),
 POI(id=9004, lat=41.8461723, lon=12.4879358, poi_type='starting_point', fee_value=0.0, jurisdiction='J2')]

In [7]:
paths = []
for ele in starting_points: 
    paths.append(t._high_value_strategy(ele))

In [8]:
paths

[[POI(id=5232181090, lat=41.89043045043945, lon=12.43865585327148, poi_type='swimming_pool', fee_value=240.3356905639248, jurisdiction='J5'),
  POI(id=11208102698, lat=41.88479232788086, lon=12.47138786315918, poi_type='sports_centre', fee_value=228.0957878683532, jurisdiction='J5'),
  POI(id=4224427195, lat=41.88856506347656, lon=12.47531795501709, poi_type='bakery', fee_value=171.00638854344817, jurisdiction='J5')],
 [POI(id=5245720404, lat=41.89482879638672, lon=12.49997806549072, poi_type='events_venue', fee_value=263.5838435718063, jurisdiction='J6'),
  POI(id=8946857225, lat=41.91931915283203, lon=12.4990062713623, poi_type='dentist', fee_value=256.0430060585064, jurisdiction='J6'),
  POI(id=6782498346, lat=41.89970397949219, lon=12.49187850952148, poi_type='sports', fee_value=229.9843252238471, jurisdiction='J6'),
  POI(id=10748877212, lat=41.89953994750977, lon=12.49194622039795, poi_type='hairdresser', fee_value=139.23176675426865, jurisdiction='J6')],
 [POI(id=9312609169, lat

In [12]:
flat_data = []
for route_idx, poi_list in enumerate(paths):
    for poi_idx, poi in enumerate(poi_list):
        flat_data.append({
            "route_id": f"R_{poi.jurisdiction}_{route_idx}",
            "jurisdiction_id": poi_idx,
            "poi_id": poi.id,
            "lat": poi.lat,
            "lon": poi.lon
        })

df_fin = pd.DataFrame(flat_data)
df_fin.to_csv("valuecsv_submission_team2.csv")